In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns
from pyfinite import ffield
import csv

# Create a class to generate waveforms

class WaveformGenerator:
    def __init__(self):
        self.pulse_width = None
        self.bandwidth = None
        self.chirp = None
        

    # Sinusoidal wave
    def sinusoidal(self, frequency, amplitude = 1, sampling_frequency=7.2):
        amplitude = float(amplitude)
        frequency = float(frequency) * 1e9  # user gives frequency in GHz
        sampling_frequency = float(sampling_frequency) * 1e9  # user gives sampling frequency in GHz
        period = 1 / frequency
        oversample = int(period * sampling_frequency)
        time = np.arange(0, period, 1 / sampling_frequency)
        wave = (amplitude / 2) * np.sin(2 * np.pi * frequency * time)

        print(f"num samples: {len(wave)}, over sample {oversample}")

        return  time, wave

    def get_taps(self, order):
        F = ffield.FField(order)
        taps = [i for i, bit in enumerate(reversed(F.ShowCoefficients(F.generator))) if bit == 1]
        print("taps: ", taps)
        return taps[:-1]  # remove x^0 term which is always 1 in primitive polynomials

    def PRBS(self, amplitude, order, repetition_rate, sampling_frequency=7.2, max_bits=None):
        amplitude = float(amplitude)
        order = int(order)
        taps = self.get_taps(order)
        sampling_frequency = float(sampling_frequency) * 7.2e9
        repetition_rate = float(repetition_rate) * 1e6

        oversample = round(sampling_frequency / repetition_rate)
        max_length = (2 ** order) - 1

        if max_bits is not None:
            length = min(max_length, int(max_bits))
        else:
            length = max_length

        while True:
            seed = np.random.randint(0, 2, size=order).tolist()
            if any(seed):
                break

        state = seed.copy()
        bits = []
        for _ in range(length):
            feedback = 0
            for t in taps:
                feedback ^= state[t]
            bits.append(state[-1])
            state = [feedback] + state[:-1]

        bits = np.array(bits)
        print(f"[DEBUG] order={order}, length={length}, oversample={oversample}, total_samples={length * oversample}")
        print("Bits:", bits[:50])
        print("Unique bit values:", np.unique(bits))

        waveform = np.repeat(bits * amplitude, oversample)
        time = np.arange(len(waveform)) / sampling_frequency

        return time, bits, waveform
    
    def chirp_calculator(self, pulse_width, bandwidth):
        chirp = bandwidth / pulse_width
        return chirp 
    def pulse_width_calculator(self, chirp, bandwidth):
        pulse_width = bandwidth / chirp
        return pulse_width
    def bandwidth_calculator(self, chirp, pulse_width):
        bandwidth = chirp * pulse_width
        return bandwidth

    def generate_lfm(self, amplitude, center_freq, sampling_freq = 7.2,
                     bandwidth=None, pulse_width=None, chirp=None):
        
        self.sampling_freq = float(sampling_freq) * 1e9  # GHz to Hz
        self.center_freq = float(center_freq) * 1e9  # GHz to Hz       


        # Validate input
        if chirp is not None and (bandwidth is not None and pulse_width is not None):
            raise ValueError("Provide either (chirp) or (bandwidth or pulse_width), not all.")
        elif chirp is not None and (bandwidth is None and pulse_width is None):
            raise ValueError("provide either bandwidth or pulse width along with chirp")
        else:
            pass
        if chirp is not None and bandwidth is not None:
            self.chirp = chirp
            self.bandwidth = bandwidth * 1e9
            self.pulse_width = self.pulse_width_calculator(chirp = self.chirp, bandwidth = self.bandwidth) 

        elif chirp is not None and pulse_width is not None:
            self.chirp = chirp
            self.pulse_width = pulse_width * 1e-9
            self.bandwidth = self.bandwidth_calculator(chirp = self.chirp, pulse_width = self.pulse_width)    
        else:
            self.bandwidth = bandwidth * 1e9
            self.pulse_width = pulse_width * 1e-9
            self.chirp = self.chirp_calculator(pulse_width= self.pulse_width, bandwidth= self.bandwidth)

        self.f0 = self.center_freq - self.bandwidth / 2
        self.f1 = self.center_freq + self.bandwidth / 2

        t = np.arange(0, self.pulse_width, 1 / self.sampling_freq)
        phase = -2 * np.pi * ((self.f0 * t) + (0.5 * self.chirp * (t ** 2)))
        I = np.cos(phase)
        Q = np.sin(phase)
        waveform = amplitude  * (I * (np.cos( 2 * np.pi * 1e9 * t)) - Q * (np.sin(2 * np.pi * 1e9 * t))) #signal.chirp(t, f0=self.f0, f1=self.f1, t1=self.pulse_width, method='linear') * amplitude
        return t, waveform

    def plot(self, time, waveform, amplitude, title="Waveform", bits=None, num_bits=50):
        plt.figure(figsize=(10, 4))
        if bits is not None:
            oversample = len(waveform) // len(bits)
            num_bits = min(num_bits, len(bits))
            num_samples = oversample * num_bits
        else:
            num_samples = len(waveform)

        plt.plot(time[:num_samples], waveform[:num_samples], label='Waveform', linewidth=1.5)

        if bits is not None:
            oversample = len(waveform) // len(bits)
            bit_times = time[:len(bits) * oversample:oversample]
            plt.step(bit_times[:num_bits], bits[:num_bits] * amplitude,
                     where='post', linestyle='--', label='Bits', alpha=0.5)

        plt.title(title + f' sample rate 7.2GHz')
        plt.xlabel("Time (ns)")
        plt.ylabel("Amplitude (V)")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

    


In [42]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns
from pyfinite import ffield
import csv
import plotly.graph_objects as go

# Create a class to generate waveforms

class WaveformGenerator:
    def __init__(self):
        self.pulse_width = None
        self.bandwidth = None
        self.chirp = None

    # Sinusoidal wave
    def sinusoidal(self, amplitude, frequency, sampling_frequency=7.2):
        amplitude = float(amplitude)
        frequency = float(frequency) * 1e9  # user gives frequency in GHz
        sampling_frequency = float(sampling_frequency) * 1e9  # user gives sampling frequency in GHz
        period = 1 / frequency
        oversample = int(period * sampling_frequency)
        time = np.arange(0, period, 1 / sampling_frequency)
        wave = (amplitude / 2) * np.sin(2 * np.pi * frequency * time)
        print(f"num samples: {len(wave)}, over sample {oversample}")

        return oversample, time, wave

    def get_taps(self, order):
        F = ffield.FField(order)
        taps = [i for i, bit in enumerate(reversed(F.ShowCoefficients(F.generator))) if bit == 1]
        print("taps: ", taps)
        return taps[:-1]  # remove x^0 term which is always 1 in primitive polynomials

    def PRBS(self, amplitude, order, repetition_rate, sampling_frequency=7.2, max_bits=None):
        amplitude = float(amplitude)
        order = int(order)
        taps = self.get_taps(order)
        sampling_frequency = float(sampling_frequency) * 7.2e9
        repetition_rate = float(repetition_rate) * 1e6

        oversample = round(sampling_frequency / repetition_rate)
        max_length = (2 ** order) - 1

        if max_bits is not None:
            length = min(max_length, int(max_bits))
        else:
            length = max_length

        while True:
            seed = np.random.randint(0, 2, size=order).tolist()
            if any(seed):
                break

        state = seed.copy()
        bits = []
        for _ in range(length):
            feedback = 0
            for t in taps:
                feedback ^= state[t]
            bits.append(state[-1])
            state = [feedback] + state[:-1]

        bits = np.array(bits)
        print(f"[DEBUG] order={order}, length={length}, oversample={oversample}, total_samples={length * oversample}")
        print("Bits:", bits[:50])
        print("Unique bit values:", np.unique(bits))

        waveform = np.repeat(bits * amplitude, oversample)
        time = np.arange(len(waveform)) / sampling_frequency

        return oversample, time, bits, waveform
    
    def generate_lfm(self, center_freq, bandwidth, pulse_width, sampling_freq = 7.2):
        
        self.sampling_freq = float(sampling_freq) * 1e9  # GHz to Hz
        self.center_freq = float(center_freq) * 1e9  # GHz to Hz
        self.pulse_width = pulse_width * 1e-9
        self.bandwidth = float(bandwidth * 1e9)
        self.k = self.bandwidth/self.pulse_width
        

        self.f0 = float(self.center_freq - self.bandwidth / 2)
        self.f1 = float(self.center_freq + self.bandwidth / 2)

        t = np.arange(0, self.pulse_width, 1 / self.sampling_freq)
        waveform = np.cos(2*np.pi * ((self.f0 * t) + (self.k/2) * (t ** 2)))   #signal.chirp(t, f0=self.f0, f1=self.f1, t1=self.pulse_width, method='linear') 

        return t, waveform

    def plot(self, time, waveform, amplitude, title="Waveform", bits=None, num_bits=50):
        fig = go.Figure()

        if bits is not None:
            oversample = len(waveform) // len(bits)
            num_bits = min(num_bits, len(bits))
            num_samples = oversample * num_bits
        else:
            num_samples = len(waveform)

        # Main waveform
        fig.add_trace(go.Scatter(
            x=time[:num_samples],
            y=waveform[:num_samples],
            mode='lines',
            name='Waveform',
            line=dict(width=2)
        ))

        # Optional bits as step plot
        if bits is not None:
            oversample = len(waveform) // len(bits)
            bit_times = time[:len(bits) * oversample:oversample]
            bit_values = bits[:num_bits] * amplitude

            # Create step-like structure manually for 'post' style
            step_x = []
            step_y = []
            for i in range(num_bits):
                step_x.extend([bit_times[i], bit_times[i] + oversample * (time[1] - time[0])])
                step_y.extend([bit_values[i], bit_values[i]])

            fig.add_trace(go.Scatter(
                x=step_x,
                y=step_y,
                mode='lines',
                name='Bits',
                line=dict(dash='dash'),
                opacity=0.5
            ))

        fig.update_layout(
            title=title + " sample rate 7.2GHz",
            xaxis_title="Time (ns)",
            yaxis_title="Amplitude (V)",
            legend=dict(x=0, y=1),
            margin=dict(l=40, r=40, t=40, b=40),
            template="plotly_white"
        )

        fig.show()
    def save_file(self, waveform_name, wave):
        filename = f"single_channel_{waveform_name}_waveform.csv"
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Ch1"])
            for value in wave:
                writer.writerow([value])
        print(f"Saved: {filename}")


In [43]:
#4634156748
import plotly.graph_objects as go
from numpy.fft import fftshift, fft
generator = WaveformGenerator()
pulse_width = 100
bandwidth = 0.25
c_freq = 1.5

sampls, time, wave = generator.sinusoidal(amplitude=1, frequency=.01)
#generator.plot(time=time, waveform=wave, amplitude=amplitude, title= f"Sinusoidal wave of frequency {frequency} MHz and ")
#num_samples, prbs_time, bits, waveform = generator.PRBS(amplitude= amplitude, order=order, repetition_rate=repetition_rate, max_bits= 100)
#generator.plot(time=prbs_time, waveform= waveform, amplitude=amplitude, bits=bits, title=f"PRBS of order {order} and repetition rate {repetition_rate} MHz and")
T, W = generator.generate_lfm( center_freq=c_freq, bandwidth=bandwidth, pulse_width=pulse_width)
generator.plot(amplitude=1, waveform=W, time=T, title=f'LFM_{c_freq}_MHz_center_freq')
wave = fftshift(fft(W))
wave = np.absolute(wave) ** 2
def fft_signal(w, iota):
        N = 2 ** np.ceil(np.log2(len(w) * 4))
        f, x = signal.periodogram(w, window='hann', nfft=N, fs=1e9 * iota, scaling='spectrum')
        f = f / 1e9
        epsilon = 1e-10
        x = 10 * (np.log10((x + epsilon) / 50e-3))
        return [f, x]
f, x = fft_signal(W, 7.2)
N = len(wave)
n = np.arange(N)
T = N/ (7.2 * 1e9)  # Convert to time in seconds
freq = n/T
fig = go.Figure(data=go.Scatter(x = freq/1e9, y=np.log10(wave/1e-3), mode='lines', name='FFT Signal'))
#limit x axis
fig.update_layout(
    xaxis=dict(
        range=[0, 3.6],  # Limit x-axis from 0 to 3.6 GHz (Nyquist)
        title="Frequency (GHz)"
    ),
    yaxis=dict(
        title="Amplitude (log10 scale)"
    ),
    title="FFT Spectrum",
    template="plotly_white"
)

fig.show()

num samples: 720, over sample 720


In [5]:
import numpy as np
import csv
import plotly.graph_objects as go
from numpy.fft import fft, fftshift

# -------- Waveform Generators --------
def generate_sine(amplitude, frequency, num_samples, fs):
    t = np.arange(num_samples) / fs
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return t, wave

def generate_prbs(amplitude, num_samples, fs):
    t = np.arange(num_samples) / fs
    bits = np.random.choice([-1, 1], size=num_samples)
    return t,  bits

def generate_lfm(center_freq, bandwidth, pulse_width, num_samples, fs):
    t = np.arange(num_samples) / fs
    k = bandwidth / pulse_width
    wave = np.sin(2 * np.pi * (center_freq * t + 0.5 * k * t**2))
    return t, wave

def generate_step_lfm(start_freq, stop_freq, step_freq, dwell_time, num_samples, fs):
    fs = fs * 1e9  # GHz → Hz
    start_freq_hz = start_freq * 1e9
    stop_freq_hz  = stop_freq * 1e9
    step_freq_hz  = step_freq * 1e9
    dwell_time_s  = dwell_time * 1e-9  # ns → s
    
    samples_per_step = int(dwell_time_s * fs)
    
    freqs = np.arange(start_freq_hz, stop_freq_hz + step_freq_hz/2, step_freq_hz)
    t_total = []
    waveform = []
    
    time_offset = 0
    for f in freqs:
        t_step = np.arange(samples_per_step) / fs + time_offset
        wave_step = np.full(samples_per_step, f)  # store frequency value directly
        t_total.append(t_step)
        waveform.append(wave_step)
        time_offset += dwell_time_s

    # Flatten
    t_total = np.concatenate(t_total)
    waveform = np.concatenate(waveform)

    # Trim if needed
    if len(waveform) > num_samples:
        t_total = t_total[:num_samples]
        waveform = waveform[:num_samples]


    return t_total, waveform


# -------- Parameters --------
num_samples = 1024
fs = 7.6e9  # 1 GHz sample rate
pulse_width = 1000 * 1e-9  # for LFM and step LFM
selected = ["sine", "step_lfm"]  # choose any combination

# -------- Generate & Combine --------
combined_wave = np.zeros(num_samples)
t = np.arange(num_samples) / fs

for wf in selected:
    if wf == "sine":
        _, wave = generate_sine(1, 10e6, num_samples, fs)
    elif wf == "prbs":
        _, wave = generate_prbs(1, num_samples, fs)
    elif wf == "lfm":
        _, wave = generate_lfm(1e9, 0.5e9, pulse_width, num_samples, fs)
    elif wf == "step_lfm":
        _, wave = generate_step_lfm(start_freq= .1, step_freq=.02, stop_freq=.5, dwell_time=100, num_samples=num_samples, fs=fs/1e9)
    combined_wave += wave

# -------- FFT using NumPy --------
wave_fft = fftshift(fft(combined_wave, n=300))
psd = np.abs(wave_fft) ** 2
freq_axis = np.linspace(-fs/2, fs/2, num_samples) / 1e9  # MHz

# -------- Plot: Time Domain --------
fig_time = go.Figure()
fig_time.add_trace(go.Scatter(
    x=t * 1e9,  # time in µs
    y=combined_wave/1e9,
    mode='lines',
    name="Combined Waveform"
))
fig_time.update_layout(
    title=f"Time Domain: {', '.join(selected)}",
    xaxis_title="Time (µs)",
    yaxis_title="Amplitude",
    template="plotly_white"
)

# -------- Plot: Frequency Domain --------
fig_freq = go.Figure()
fig_freq.add_trace(go.Scatter(
    x=freq_axis,
    y=10 * np.log10(psd + 1e-12),  # dB scale
    mode='lines',
    name="FFT Magnitude"
))
fig_freq.update_layout(
    title="FFT Spectrum",
      
    xaxis_title="Frequency (GHz)",
    yaxis_title="Power (dB)",
    template="plotly_white"
)

# -------- Show Plots --------
fig_time.show()
fig_freq.show()

# -------- Save CSV --------
with open("combined_waveform.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Time (s)", "Amplitude"])
    for ti, amp in zip(t, combined_wave):
        writer.writerow([ti, amp])

print("Saved to combined_waveform.csv")


Saved to combined_waveform.csv
